In [10]:
import cv2
import pytesseract

# Tentukan path ke executable Tesseract (ganti dengan path yang sesuai pada sistem Anda)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Load image
image = cv2.imread('image-open-cv/logo.png')  # Ganti dengan nama file gambar Anda

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply GaussianBlur to reduce noise
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# Use Canny edge detection to detect edges
edges = cv2.Canny(blurred, 100, 200)

# Find contours in the edge-detected image
contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Sort contours by area to find the largest one (potential plate)
contours = sorted(contours, key=cv2.contourArea, reverse=True)

# Iterate through contours to find the one that resembles a plate
plate_contour = None
for contour in contours:
    # Get the bounding box of the contour
    x, y, w, h = cv2.boundingRect(contour)
    
    # Check aspect ratio of the bounding box (it should be rectangular)
    aspect_ratio = w / float(h)
    if aspect_ratio > 2 and aspect_ratio < 5:  # Plat nomor biasanya memiliki rasio panjang-lebar seperti ini
        plate_contour = contour
        break

# If a plate contour is found, extract it
if plate_contour is not None:
    x, y, w, h = cv2.boundingRect(plate_contour)
    plate_image = image[y:y+h, x:x+w]

    # Convert the plate image to grayscale and apply thresholding
    plate_gray = cv2.cvtColor(plate_image, cv2.COLOR_BGR2GRAY)
    _, plate_thresh = cv2.threshold(plate_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Use Tesseract OCR to read the plate number
    plate_text = pytesseract.image_to_string(plate_thresh, config='--psm 8')  # PSM 8 is good for OCR in a block of text

    # Display the result
    print("Detected Plate Number: ", plate_text)

    # Show the detected plate image (optional)
    cv2.imshow("Detected Plate", plate_image)
    cv2.waitKey(0)

else:
    print("Plat nomor tidak ditemukan!")

# Close all OpenCV windows
cv2.destroyAllWindows()


Plat nomor tidak ditemukan!
